# Your Title Here

**Name(s)**: Landon Marchetti and Lyn Mansfield

**Website Link**: (your website link)

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from matplotlib.backends.backend_pdf import PdfPages

import plotly.express as px
pd.options.plotting.backend = 'plotly'

from dsc80_utils import * # Feel free to uncomment and use this.

In [2]:
%load_ext autoreload
%autoreload 2

## Step 1: Introduction

# TODO

Which legends are the most meta-defining? Difference in win rate between when it's banned vs when it's not

How important is the first kill of a game? 
- not important at all actually lol, the winrates are all the same no matter what

Which characters are the most banned? Which characters are the best?

What archetype of character's performance matters the most?

How important is early success?

## Step 2: Data Cleaning and Exploratory Data Analysis

What are "partial" rows missing from their data? <== me!

In [195]:
# TODO

from pathlib import Path

dataset_path = Path("./data/2025_LoL_clean_stats.csv")
# /data/2025_LoL_clean_stats.csv
data = pd.read_csv(dataset_path)
data

,Unnamed: 0,gameid,datacompleteness,url,...,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm
0,0,LOLTMNT03_179647,complete,NaN,...,0,-1,-1,8.82
1,1,LOLTMNT03_179647,complete,NaN,...,132,-1,-1,5.39
2,2,LOLTMNT03_179647,complete,NaN,...,0,-1,-1,7.88
...,...,...,...,...,...,...,...,...,...
60177,60177,LOLTMNT06_120342,complete,NaN,...,0,-1,-1,1.10
60178,60178,LOLTMNT06_120342,complete,NaN,...,159,-1,-1,34.30
60179,60179,LOLTMNT06_120342,complete,NaN,...,222,-1,-1,36.61


In [182]:
# for boolean columns, give special treatment
boolean_columns = ['firstblood', 'firstbloodkill', 'firstbloodassist', 'firstbloodvictim']
def transform_int_to_bool(x):
    if x == 1:
        return True
    if x == 0:
        return False
for column in boolean_columns:
    data[column] = data[column].apply(lambda x: transform_int_to_bool(x) if x >= 0 else np.nan)

# clearly label if players won or lost
data['result'] = data['result'].apply(lambda x: 'Won' if x == 1 else 'Lost')

data

,gameid,datacompleteness,url,league,...,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm
0,LOLTMNT03_179647,complete,NaN,LFL2,...,0,-1,-1,8.82
1,LOLTMNT03_179647,complete,NaN,LFL2,...,132,-1,-1,5.39
2,LOLTMNT03_179647,complete,NaN,LFL2,...,0,-1,-1,7.88
...,...,...,...,...,...,...,...,...,...
60177,LOLTMNT06_120342,complete,NaN,CD,...,0,-1,-1,1.10
60178,LOLTMNT06_120342,complete,NaN,CD,...,159,-1,-1,34.30
60179,LOLTMNT06_120342,complete,NaN,CD,...,222,-1,-1,36.61


In [186]:
data.to_csv("2025_LoL_clean_stats.csv")

In [188]:
type_frequencies = data.dtypes.value_counts()
type_frequencies.index = type_frequencies.index.map(str)
fig = px.bar(type_frequencies, 
	labels={'index': 'Data type', 'value': 'Frequency'}
	)
fig.update_layout(
    title={
        'text': 'Frequencies of Relevant Data Types',
        'font': {'size': 24}
    }
)
fig.show()

In [187]:
num_missing = data.apply(pd.isna).sum()
num_missing = num_missing / data.shape[0] * 100

fig = px.bar(num_missing[num_missing != 0].sort_values(), 
	labels={'index': 'Column name', 'value': '% missing'},
	title='Data missing from partial rows'
	)
fig.show()

## Step 3: Assessment of Missingness

In [124]:
# TODO

def univariate_plots(df, title):
    pdf_pages = PdfPages(f'univariate_plots_{title}.pdf')

    plt.figure()

    for column in df.columns:
        fig, ax = plt.subplots()
        ax.hist(df[column], bins=200)
        ax.set_title(f'Graph of {column}')
        pdf_pages.savefig(fig)
        plt.close(fig)
    
    pdf_pages.close()

def bivariate_plot(df, categorical_column, target_column, title):
    pdf_pages = PdfPage(f'bivariate_plots_{title}.pdf')

    plt.figure()
    
    for column in df.columns:
        fig, ax = plt.subplots()
        ax.hist(df[column])
        ax.set_title(f'Graph of {column}')
        pdf_pages.savefig(fig)
        plt.close(fig)
    
    pdf_pages.close()

In [150]:
player_data = data[data['damageshare'].notna()]

In [151]:
fig = player_data['dpm'].hist()
fig.show()

In [152]:
fig = player_data['damagetakenperminute'].hist()
fig.show()

In [154]:
fig = player_data['damagemitigatedperminute'].hist()
fig.show()

## Step 4: Hypothesis Testing

In [217]:
# TODO

player_rows = data[data['playername'].notna()]
player_rows

,Unnamed: 0,gameid,datacompleteness,url,...,monsterkills,monsterkillsownjungle,monsterkillsenemyjungle,cspm
0,0,LOLTMNT03_179647,complete,NaN,...,0,-1,-1,8.82
1,1,LOLTMNT03_179647,complete,NaN,...,132,-1,-1,5.39
2,2,LOLTMNT03_179647,complete,NaN,...,0,-1,-1,7.88
...,...,...,...,...,...,...,...,...,...
60175,60175,LOLTMNT06_120342,complete,NaN,...,26,-1,-1,11.08
60176,60176,LOLTMNT06_120342,complete,NaN,...,10,-1,-1,10.44
60177,60177,LOLTMNT06_120342,complete,NaN,...,0,-1,-1,1.10


In [219]:
player_rows['result'] = player_rows['result'].apply(lambda x: 1 if x == "Won" else 0)
champion_win_rates = player_rows.groupby("champion")['result'].mean()

## Step 5: Framing a Prediction Problem

In [ ]:
# TODO
Use one-hot encoding!

## Step 6: Baseline Model

In [ ]:
# TODO

## Step 7: Final Model

In [ ]:
# TODO

## Step 8: Fairness Analysis

In [ ]:
# TODO